### **Análise dos dados**

Segue abaixo a análise dos dados inicialmente fornecidos.

Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

Inicializando os dados

In [ ]:
df_resultados = pd.read_csv('data/RESULTADOS_04_06_2024_full_teste.csv', compression='gzip')
df_resultados2 = pd.read_csv('data/RESULTADOS_02_03_2024_full_teste.csv', compression='gzip')
df_falhas = pd.read_csv('data/FALHAS_04_05_06_2024.csv')

Checando os dados

In [ ]:
# Remover coluna errada
df_resultados = df_resultados.drop(columns=['Unnamed: 0'], axis=1)
df_resultados2 = df_resultados2.drop(columns=['Unnamed: 0'], axis=1)



In [ ]:
df_falhas = df_falhas.drop(columns=['Unnamed: 0'], axis=1)

Arrumar os dados de falha

In [ ]:
df_falhas.columns = df_falhas.iloc[0]

df_falhas = df_falhas.drop(df_falhas.index[0])

Checar ambos DFs

In [ ]:
df_falhas.head()

Analisando os tipos de dados de falhas

In [ ]:
df_falhas.describe()

In [ ]:
df_resultados.head(20)

Após está analise, creio que os dados de ambas as tabelas que não serão utilizados tanto no treinamento quanto teste do modelo são:

#### Falhas:

- KNR
- Estação
- Data

#### Resultados:

- KNR
- Name
- Data

OBS(Futuramente a coluna Data pode ser utilizada para criar novas features)
OBS2(Colunas KNR e Name serão removidas apenas antes do treinamento do modelo)

## Preparação dos dados

Inicialmente, os dados serão integrados em uma única tabela

In [ ]:
# Remoção das linhas com NaN

df_falhas = df_falhas.dropna()
df_resultados = df_resultados.dropna()
df_resultados2 = df_resultados2.dropna()

In [ ]:
df_resultados.head()

Convertendo a coluna DATA para um formato melhor

In [ ]:
df_resultados.info()

In [ ]:
# Salvando os dados em "parquet" é mais rápido para leitura

df_resultados.to_parquet('data/df_resultados.parquet', index=False)
df_resultados2.to_parquet('data/df_resultados2.parquet', index=False)

In [ ]:
# Limpando colunas que não serão usadas agora

colunas = ['UNIT', 'VALUE_ID', 'VALUE']

df_resultados2 = df_resultados2.drop(columns=colunas, axis=1)
df_resultados = df_resultados.drop(columns=colunas, axis=1)

In [ ]:
# Transformando os DFs de resultados em apenas um DF de resultados maior

df = pd.concat([df_resultados, df_resultados2])

In [ ]:
# Transformando data em datetime do pandas

df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')

Segue abaixo o código para a preparação dos dados

In [ ]:
def aggregate_by_id(df, id_value):
    subset = df[df['ID'] == id_value]
    return subset.groupby('KNR').agg(
        NAME=('NAME', 'count'),  # Conta total de NAME
        SOK=('STATUS', lambda x: (x == 10).sum()),
        SNOK=('STATUS', lambda x: (x == 13).sum()),
        DATA=('DATA', lambda x: (x.max() - x.min()).total_seconds() / (3600*24))
    ).rename(columns={
        'NAME': f'ID{id_value}NAME',
        'SOK': f'ID{id_value}SOK',
        'SNOK': f'ID{id_value}SNOK',
        'DATA': f'ID{id_value}DATA'
    })

# Agregando para cada ID
id1 = aggregate_by_id(df, 1)
id2 = aggregate_by_id(df, 2)
id718 = aggregate_by_id(df, 718)

# Combinando os resultados em um único DataFrame
final_df = id1.join(id2, on='KNR', how='outer').join(id718, on='KNR', how='outer').reset_index()

# Adicionando UNIQUE_ID com valores incrementais

# Reordenando as colunas para o formato desejado
final_df = final_df[['KNR','ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK', 'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA']]

# Exibindo o resultado
print(final_df)

In [ ]:
# Colocar 0 no lugar de NaN para evitar problemas
final_df = final_df.fillna(0)

In [ ]:
final_df.head(10)

In [308]:
# Checar o tamanho do DataFrame

final_df.shape

(35317, 13)

Arrumando o DF de Falhas


In [ ]:
# Deixa todas as falhas em letra maiuscula
df_falhas['FALHA'] = df_falhas['FALHA'].str.upper()

In [ ]:
# Checa o DataFrame
df_falhas.head()

In [ ]:
# Removendo as colunas que não serão utilizadas na predição

colunas_to_remove = ['MODELO', 'COR', 'MOTOR', 'ESTACAO', 'USUARIO', 'HALLE', 'DATA']

df_falhas = df_falhas.drop(columns=colunas_to_remove, axis=1)

É possível entregarem o HALLE da tabela Resultados? **PERGUNTA PARA O CLIENTE**

In [ ]:
# Remove todas as linhas com KNR repetido
df_falhas_unique = df_falhas.drop_duplicates(subset=['KNR'])

# Checa o DataFrame
df_falhas_unique.head()


In [ ]:
# Muda os valores da coluna de Falha para 1
df_falhas_unique['FALHA'] = 1

In [ ]:
# Checa o DataFrame
df_falhas_unique.head()

Dando merge do Falhas com Resultado

In [ ]:
# Realizando o merge dos dataframes df_falhas e final_df com base na coluna 'KNR'
merged_df = pd.merge(final_df, df_falhas_unique, on='KNR', how='left')

In [ ]:
# Analisa o tamanho do DF final
merged_df.shape

In [ ]:
# Adiciona 0 em todos os NaN
merged_df = merged_df.fillna(0)

In [ ]:
merged_df.head()

Normalização e treinamento do modelo

In [ ]:
# Selecionando apenas as colunas específicas para normalização
cols_to_normalize = ['ID1NAME','ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK', 'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA']

# Inicializando o MinMaxScaler
scaler = MinMaxScaler()

# Aplicando a normalização
merged_df[cols_to_normalize] = scaler.fit_transform(merged_df[cols_to_normalize])

# Exibindo o dataframe normalizado
merged_df.head()

In [ ]:
# Separando as features (X) e o target (y)
X = merged_df.drop(columns=['FALHA', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = merged_df['FALHA']

In [ ]:
# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Inicializando o classificador KNN
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)

In [ ]:
# Performance no conjunto de treinamento
train_predictions = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

# Performance no conjunto de teste
test_predictions = knn.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

print("Acurácia no Treinamento:", train_accuracy)
print("Acurácia no Teste:", test_accuracy)

In [ ]:
# Analisa quantos valores são 1 e quantos 0 das falhas
print(y_train.value_counts())